In [1]:
import os
%pwd

'c:\\Users\\Sasu4\\SHIP_Classification_using_Resnet\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\Sasu4\\SHIP_Classification_using_Resnet'

In [86]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int
    params_classes: int
    CLASS_NAMES:list
    
    
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
    freeze_all: bool
    freeze_till: int

In [52]:
class PrepareBaseModel:
  def __init__(self, config):
        self.config = config
        self.model=None
        self.optimizer = None  # Initialize as needed
        self.loss_fn = None  # Initialize as needed

In [7]:
import torch as torch 

# Load the entire model
model = torch.load("artifacts/training/model.pth", weights_only=True)

In [11]:
from Ship_Classifier.constants import *
from Ship_Classifier.utils.common import read_yaml,create_directories
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from pathlib import Path

In [94]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.CLASS_NAMES=['Cargo', 'Cruise', 'Carrier', 'Military', 'Tanker']
        create_directories([self.config.artifacts_root])
        
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
        path_of_model=Path("artifacts/training/model.pth"),  # Updated model path to PyTorch format
        training_data=Path("artifacts/data_ingestion/extracted_data/Images"),  # Update path to your training images
        all_params=self.params,
        params_image_size=self.params.IMAGE_SIZE,
        params_batch_size=self.params.BATCH_SIZE,
        CLASS_NAMES=['Cargo', 'Cruise', 'Carrier', 'Military', 'Tanker'],  # Add class names
        params_classes=self.params.CLASSES
    )
        return eval_config   
    
    
     
    

In [89]:
from urllib.parse import urlparse
from torchvision import transforms, datasets,models
from torch.utils.data import DataLoader
import torch.nn as nn

In [103]:
from Ship_Classifier import config
from Ship_Classifier.utils.common import save_json
from Ship_Classifier.components.prepare_base_model import PrepareBaseModel

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
    def _valid_loader(self):
        # Define transformations
        transform = transforms.Compose([
            transforms.Resize(self.config.params_image_size[:-1]),  # Resize images to the desired size
            transforms.ToTensor(),  # Convert images to PyTorch tensors
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
        ])

        # Load validation dataset
        validation_dataset = datasets.ImageFolder(
            root=self.config.training_data,  # Path to the validation data
            transform=transform
        )
        # Display the number of images in each folder (class)
        class_counts = {class_name: 0 for class_name in validation_dataset.classes}
        for _, label in validation_dataset:
            class_name = validation_dataset.classes[label]
            class_counts[class_name] += 1
        
        for class_name, count in class_counts.items():
            print(f"Found {count} images belonging to class '{class_name}'")

        # Create data loader
        self.valid_loader = DataLoader(
            validation_dataset,
            batch_size=self.config.params_batch_size,
            shuffle=False
        )

    @staticmethod
    def load_model(path: str,params_classes:int) -> nn.Module:
        # Load the saved PyTorch model
        model=models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        model.fc = nn.Linear(model.fc.in_features, params_classes)
       
        state_dict = torch.load(path)
        # Load the state dictionary into the model
        model.load_state_dict( state_dict)
        model.eval()  # Set the model to evaluation mode
        return model

    def evaluation(self):
        # Load the model
        self.model = self.load_model(self.config.path_of_model,self.config.params_classes).to(self.device)

        # Prepare the validation data loader
        self._valid_loader()
        
        

        # Set up evaluation metrics
        criterion = nn.CrossEntropyLoss()
        total_loss = 0.0
        correct_predictions = 0
        

        # Evaluate the model
        with torch.no_grad():
            for inputs, labels in self.valid_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                total_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                correct_predictions += (predicted == labels).sum().item()

        # Calculate average loss and accuracy
        self.average_loss = total_loss / len(self.valid_loader.dataset)
        self.accuracy = correct_predictions / len(self.valid_loader.dataset)

        self.score = {"loss": self.average_loss, "accuracy": self.accuracy}

    def save_score(self):
        
        
        # Save the scores as a JSON file
        save_json(path=Path("scores.json"), data=self.score)

In [104]:
try:
    # Initialize the configuration manager
    config = ConfigurationManager()

    # Get the validation configuration
    val_config = config.get_validation_config()

    # Create an Evaluation instance with the validation configuration
    evaluation = Evaluation(val_config)

    # Perform the evaluation
    evaluation.evaluation()

    # Save the evaluation score
    evaluation.save_score()

except Exception as e:
    # Raise the exception if any errors occur
    raise e

[2024-08-27 19:17:54,220: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-27 19:17:54,224: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-27 19:17:54,226: INFO: common: created directory at: artifacts]


C:\Users\Sasu4\AppData\Local\Temp\ipykernel_31632\2995088565.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path)


Found 99 images belonging to class 'Cargo'
Found 106 images belonging to class 'Carrier'
Found 108 images belonging to class 'Cruise'
Found 108 images belonging to class 'Military'
Found 118 images belonging to class 'Tanker'
[2024-08-27 19:18:34,825: INFO: common: json file saved at: scores.json]
